In [33]:
import sys
import time 
import numpy as np
from tank import *
import vrep

import skfuzzy
from skfuzzy import control as ctrl

In [34]:
def drive_carefully(distance, tank):
    distance_fuzzy = ctrl.Antecedent(np.arange(0, 9, 0.01), 'distance_fuzzy')
    current_speed = ctrl.Antecedent(np.arange(0, 10, 0.01), 'current_speed')
    new_speed = ctrl.Consequent(np.arange(0, 10, 0.01), 'new_speed')

    distance_fuzzy.automf(7)
    current_speed.automf(3)
    new_speed.automf(3)

    distance_fuzzy['dismal'] = skfuzzy.trimf(distance_fuzzy.universe, [0, 0, 0.25])
    distance_fuzzy['poor'] = skfuzzy.trimf(distance_fuzzy.universe, [0.25, 0.75, 1.25])
    distance_fuzzy['mediocre'] = skfuzzy.trimf(distance_fuzzy.universe, [1, 3, 4.5])
    new_speed['poor'] = skfuzzy.trimf(new_speed.universe, [0, 0.5, 1])

    # dismal poor mediocre average decent good excellent
    rules = (
    ctrl.Rule((distance_fuzzy['excellent'] | distance_fuzzy['good']) & (current_speed['poor'] | current_speed['average']), new_speed['good']),
    ctrl.Rule((distance_fuzzy['decent']) & (current_speed['poor'] | current_speed['average']), new_speed['good']),
    ctrl.Rule(distance_fuzzy['average'] & (current_speed['good'] | current_speed['average']), new_speed['good']),
    ctrl.Rule((distance_fuzzy['mediocre']) & (current_speed['good'] | current_speed['average']), new_speed['average']),
    ctrl.Rule((distance_fuzzy['poor']) & (current_speed['good'] | current_speed['average']), new_speed['poor']),
    ctrl.Rule(distance_fuzzy['dismal'], new_speed['poor']),
    )

    driving_ctrl = ctrl.ControlSystem(rules)
    driving = ctrl.ControlSystemSimulation(driving_ctrl)

    driving.input['distance_fuzzy'] = distance
    driving.input['current_speed'] = tank.readVelocity()
    driving.compute()

    return driving.output['new_speed']

In [35]:
def drive_aggro(distance, tank):
    distance_fuzzy = ctrl.Antecedent(np.arange(0, 9, 0.01), 'distance_fuzzy')
    current_speed = ctrl.Antecedent(np.arange(0, 10, 0.01), 'current_speed')
    new_speed = ctrl.Consequent(np.arange(0, 10, 0.01), 'new_speed')

    distance_fuzzy.automf(7)
    current_speed.automf(3)
    new_speed.automf(3)

    distance_fuzzy['dismal'] = skfuzzy.trimf(distance_fuzzy.universe, [0, 0, 0.25])
    distance_fuzzy['poor'] = skfuzzy.trimf(distance_fuzzy.universe, [0.25, 0.75, 1.25])
    distance_fuzzy['mediocre'] = skfuzzy.trimf(distance_fuzzy.universe, [1, 3, 4.5])

    current_speed['poor'] = skfuzzy.trimf(current_speed.universe, [0, 0.5, 1])
    current_speed['average'] = skfuzzy.trimf(current_speed.universe, [0.5, 5, 9.5])
    current_speed['good'] = skfuzzy.trimf(current_speed.universe, [9, 9.5, 10])

    new_speed['poor'] = skfuzzy.trimf(new_speed.universe, [0, 0.5, 1])
    new_speed['average'] = skfuzzy.trimf(new_speed.universe, [0.5, 5, 9.5])
    new_speed['good'] = skfuzzy.trimf(new_speed.universe, [9, 9.5, 10])

    # current_speed.view()
    rules = (
    ctrl.Rule((distance_fuzzy['excellent'] | distance_fuzzy['good'] | distance_fuzzy['decent']) & (current_speed['poor'] | current_speed['average']), new_speed['good']),
    ctrl.Rule((distance_fuzzy['decent'] | distance_fuzzy['average'] | distance_fuzzy['mediocre']) & (current_speed['average'] | current_speed['good']), new_speed['average']),
    ctrl.Rule((distance_fuzzy['poor']) & (current_speed['average'] | current_speed['good']), new_speed['poor']),
    ctrl.Rule(distance_fuzzy['dismal'], new_speed['good']),
    )
    # distance_fuzzy.view()

    driving_ctrl = ctrl.ControlSystem(rules)
    driving = ctrl.ControlSystemSimulation(driving_ctrl)

    driving.input['distance_fuzzy'] = distance
    driving.input['current_speed'] = tank.readVelocity()
    driving.compute()

    return driving.output['new_speed']

In [36]:
from time import sleep

vrep.simxFinish(-1) # closes all opened connections, in case any prevoius wasnt finished
clientID=vrep.simxStart('127.0.0.1',19999,True,True,5000,5) # start a connection

if clientID!=-1:
    print ("Connected to remote API server")
else:
    print("Not connected to remote API server")
    sys.exit("Could not connect")

#create instance of Tank
tank=Tank(clientID)

# get handle to proximity sensor
err_code,ps_handle = vrep.simxGetObjectHandle(clientID,"Proximity_sensor", vrep.simx_opmode_blocking)

t = time.time()

while (time.time()-t)<100: # 10 seconds of communitation
    #read values from proximity sensor
    err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,ps_handle,vrep.simx_opmode_streaming)
    distance = np.linalg.norm(detectedPoint)

    new_speed = drive_carefully(distance, tank)
    # new_speed = drive_aggro(distance, tank)
    if new_speed < 0.5:
        tank.forward(0)
        # sleep(10)
        break
    else:
        tank.forward(new_speed)

    print(f'\r{new_speed, tank.readVelocity(), distance}', end='')
    # avoid collision

vrep.simxStopSimulation(clientID,vrep.simx_opmode_oneshot) # stop the simulation in vrep

Connected to remote API server
(0.5000000000000006, 0.5000000000000006, 0.9971662339446348)

KeyboardInterrupt: 